In [ ]:
# https://www.geeksforgeeks.org/scrape-google-search-results-using-python-beautifulsoup/
# https://practicaldatascience.co.uk/data-science/how-to-scrape-google-search-results-using-python

In [ ]:
# !pip install requests_html

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/Documents/github/just-a-recipe-generator

/Users/EGK1/Documents/github/just-a-recipe-generator


In [3]:
import requests
import json
import bs4
import pickle
from bs4 import BeautifulSoup
import urllib
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

from src.data.web_scraping import get_source
from src.data.scrape_glutenfreeonashoestring import scrape_shoestring_page, scrape_all_shoestring_pages
from src.data.pickling import save_pickle, load_pickle

In [4]:
all_recipes = scrape_all_shoestring_pages(3)


Retrieved 31 links


Retrieved 31 links


Retrieved 31 links

Saving pickle to data/raw/shoestring_recipes.pickle...


In [4]:
all_recipes = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [5]:
len(all_recipes)

51

In [10]:
all_recipes

{'https://glutenfreeonashoestring.com/gluten-free-pumpkin-cookies/': {'Gluten Free Pumpkin Cookies Recipe | Soft &amp; Cakey Fall Cookies': {'@context': 'http://schema.org/',
   '@type': 'Recipe',
   'name': 'Gluten Free Pumpkin Cookies Recipe | Soft &amp; Cakey Fall Cookies',
   'author': {'@type': 'Person', 'name': 'Nicole Hunn'},
   'description': 'Make these gluten free pumpkin cookies to welcome cooler weather. Soft and cakey, these gf pumpkin cookies are wonderfully spiced and the perfect fall treat.',
   'datePublished': '2022-10-05T12:58:00+00:00',
   'image': ['https://glutenfreeonashoestring.com/wp-content/uploads/2022/10/Stack-of-pumpkin-cookies-600x600-1.jpg',
    'https://glutenfreeonashoestring.com/wp-content/uploads/2022/10/Stack-of-pumpkin-cookies-600x600-1-500x500.jpg',
    'https://glutenfreeonashoestring.com/wp-content/uploads/2022/10/Stack-of-pumpkin-cookies-600x600-1-500x375.jpg',
    'https://glutenfreeonashoestring.com/wp-content/uploads/2022/10/Stack-of-pumpkin-

## Scrape all recipes

Need to use selenium or come up with different urls

### Not using below

In [ ]:
response = get_source(url)

In [ ]:
html = requests.get(url)
soup = BeautifulSoup(html.text, 'lxml')

In [ ]:
soup.find("div", {"class": "facetwp-facet facetwp-facet-pagination facetwp-type-pager"})

In [ ]:
soup.find("div", {"class": "facetwp-pager"})

In [ ]:
soup_text = str(soup)

In [ ]:
type(soup_text)

In [ ]:
"facetwp-facet" in soup_text

In [ ]:
len(soup_text) / 200

In [ ]:
soup_list = []
beginning = 0
end = 600
interval = 200

while beginning < len(soup_text):
    start = beginning + interval
    soup_list.append(soup_text[beginning:start])
    beginning = beginning + interval

In [ ]:
len(soup_list)

In [ ]:
for item in soup_list:
    if "pagination facetwp-type-pager" in item:
        item_index = soup_list.index(item)
        print(item_index, item)

In [ ]:
soup_list[503:505]

In [ ]:
709*200

In [ ]:
pagination_section = soup_text[100600:101000]
print(pagination_section)

In [ ]:
pages_section = soup_text[141300:142300]
print(pages_section)

In [ ]:
for item in soup_list:
    if "54" in item:
        item_index = soup_list.index(item)
        print(item_index, item)

In [ ]:
# https://betterprogramming.pub/how-to-scrape-multiple-pages-of-a-website-using-a-python-web-scraper-4e2c641cff8